In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from pymodbus.client import ModbusTcpClient


cam = 1

#value = 0
#ip_address = 192.168.1.1
#port = 100
#register = M0

def send_to_plc(value, ip_address, port, register):
    """
    Sends the prediction value to a PLC using Modbus TCP.
    
    Args:
        value (int): Value to send (1 for defective, 0 for non-defective).
        ip_address (str): IP address of the PLC.
        port (int): Port number for Modbus communication.
        register (int): Modbus register to write the value to.
    """
    client = ModbusTcpClient(ip_address, port=port)
    if not client.connect():
        print("Unable to connect to PLC")
        return False

    # Write the value to the Modbus register
    result = client.write_register(register, value)
    client.close()
    return result.isError() == False

def process_frame(frame, target_size):
    """
    Processes a webcam frame for model prediction.
    
    Args:
        frame (np.array): Captured frame from the webcam.
        target_size (tuple): Target size for resizing the frame (width, height).

    Returns:
        np.array: Preprocessed image ready for model prediction.
    """
    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

def predict_frame(frame, model, target_size):
    """
    Predicts the class of a single frame using the model.

    Args:
        frame (np.array): Captured frame from the webcam.
        model: Loaded Keras model.
        target_size (tuple): Target size for resizing the frame.

    Returns:
        str: Predicted class label.
        float: Confidence score.
    """
    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        return "Non-Defective", prediction
    else:
        return "Defective", prediction

def run_webcam_with_model(model_path, target_size=(640, 480)):
    """
    Opens the webcam, captures frames, and processes them using the model.

    Args:
        model_path (str): Path to the saved Keras model.
        target_size (tuple): Target size for resizing frames (width, height).
    """
    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Open the webcam
    cap = cv2.VideoCapture(cam)
    if not cap.isOpened():
        raise IOError("Cannot open the webcam")

    print("Webcam opened. Press 'ESC' to exit.")
    try:
        while True:
            # Capture frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Process and predict
            feedback, confidence = predict_frame(frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            prob = abs(round(confidence,2)- 0.5)*200

            # Display feedback on the frame
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {prob:.0f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show the frame in a window
            cv2.imshow("Webcam Feed", frame)

            if feedback == "defective":
                value = 0
            else:
                value = 1

            print(value)

            send_to_plc(1, '192.168.1.111', 502, 1)

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for ESC
                print("Exiting...")
                break

    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the script
if __name__ == "__main__":
    model_path = "my_model_8.0.keras"  # Path to your saved model
    run_webcam_with_model(model_path, target_size=(640, 480))


Loading model...
Model loaded successfully.
Webcam opened. Press 'ESC' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Raw Prediction: 0.0002158207498723641
Prediction: Defective (Confidence: 0.00)
1


Connection to (192.168.1.111, 502) failed: timed out


Unable to connect to PLC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
Raw Prediction: 2.5629296942497604e-06
Prediction: Defective (Confidence: 0.00)
1


Connection to (192.168.1.111, 502) failed: timed out


Unable to connect to PLC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
Raw Prediction: 1.9975419718321064e-07
Prediction: Defective (Confidence: 0.00)
1


...

Unable to connect to PLC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step
Raw Prediction: 3.0551447594007186e-07
Prediction: Defective (Confidence: 0.00)
1


Connection to (192.168.1.111, 502) failed: timed out


Unable to connect to PLC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step
Raw Prediction: 4.147290383116342e-06
Prediction: Defective (Confidence: 0.00)
1


Connection to (192.168.1.111, 502) failed: timed out


Unable to connect to PLC
Exiting...
